In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from pydantic import BaseModel, Field
import requests
import wikipedia
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS

class WikipediaInput(BaseModel):
    query: str = Field(..., description="The topic to search on Wikipedia")

@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str) -> str:
    """Search for a topic on Wikipedia and return a summary."""
    try:
        return wikipedia.summary(query, sentences=5)
    except Exception as e:
        return f"Wikipedia error: {str(e)}"

class DuckDuckGoInput(BaseModel):
    query: str = Field(..., description="The topic to search using DuckDuckGo")

@tool(args_schema=DuckDuckGoInput)
def search_duckduckgo(query: str) -> str:
    """Search DuckDuckGo and return the top result URL."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=1))
        if results:
            return results[0]["href"]
        else:
            return "No results found."

class ScrapeInput(BaseModel):
    url: str = Field(..., description="The URL of the page to extract content from.")

@tool(args_schema=ScrapeInput)
def scrape_website(url: str) -> str:
    """Extract text content from a given webpage URL."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text(separator="\n")[:1000]
    except Exception as e:
        return f"Scraping error: {str(e)}"

class SaveInput(BaseModel):
    text: str = Field(..., description="The research text to save to file.")

@tool(args_schema=SaveInput)
def save_to_file(text: str) -> str:
    """Save text content to a .txt file."""
    filename = "xz_backdoor_research.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    return f"Saved research to {filename}"

tools = [search_wikipedia, search_duckduckgo, scrape_website, save_to_file]

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3) 

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research assistant who uses tools to gather information, extract website content, and save it to a file."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [7]:

query = "Research about the XZ backdoor"
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'XZ backdoor'}`


In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.
Invoking: `save_to_file` with `{'text': 'XZ Backdoor Research:\n\nIn 

{'input': 'Research about the XZ backdoor',
 'output': 'I have researched the XZ backdoor and saved the information to a file named `xz_backdoor_research.txt`. If you need any further information or assistance, feel free to ask!'}